In [4]:
import os
import argparse
import time
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim


class Args:
    def __init__(self):
        self.method = 'dopri5'
        self.data_size=1000
        self.batch_time=10
        self.batch_size=20
        self.niters=2000
        self.test_freq=20
        self.viz=True
        self.gpu=0
        self.adjoint=True

args = Args()

In [5]:
if args.adjoint:
    from torchdiffeq import odeint_adjoint as odeint
else:
    from torchdiffeq import odeint
    
device = torch.device('cuda:' + str(args.gpu) if torch.cuda.is_available() else 'cpu')

true_y0 = torch.tensor([[2., 0.]])
t = torch.linspace(0., 25., args.data_size)
true_A = torch.tensor([[-0.1, 2.0], [-2.0, -0.1]])


class Lambda(nn.Module):

    def forward(self, t, y):
        return torch.mm(y**3, true_A)


with torch.no_grad():
    true_y = odeint(Lambda(), true_y0, t, method='dopri5')

In [9]:
true_y.shape

torch.Size([1000, 1, 2])

In [10]:
def get_batch():
    s = torch.from_numpy(np.random.choice(np.arange(args.data_size - args.batch_time, dtype=np.int64), args.batch_size, replace=False))
    batch_y0 = true_y[s]  # (M, D)
    batch_t = t[:args.batch_time]  # (T)
    batch_y = torch.stack([true_y[s + i] for i in range(args.batch_time)], dim=0)  # (T, M, D)
    return batch_y0, batch_t, batch_y